## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
import cv2
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Mon Jun  3 13:30:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **12.02.2012** with version **YOLOv8.0.28**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [4]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.0/78.2 GB disk)


In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -e .

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [5]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference using a pre-trained model
Put your images in the `Images` (in `.jpg` format) folder and run the cell below

In [12]:
def get_mask(predict, filename, tag):
    for result in predict:
      if result.masks == None:
        print(filename,"in",tag, "is MASLA")
        return np.zeros((256, 256), dtype=np.uint8)
        break
      masks = result.masks.cpu().numpy()
    index = (len(masks))
    hey = 0
    BIG_LIST = list()

    for i in range(index):
      if predict[0].masks == None:
        break
      a = (predict[0].masks.data[i].cpu().numpy() * 255).astype("uint8")
      hey+=a
    return hey

model = YOLO("Dataset1_NON_AUG.pt")
folder_path = "Images"
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.exists(file_path):
        predict_1_NON_AUG = model.predict(source=file_path, save=False, save_txt=False, boxes=False)
        cv2.imwrite(fr"Masks/{filename[:-4]}.jpg",get_mask(predict_1_NON_AUG, filename, "DATA_1_NON_AUG"))
        print(filename," has been saved to Masks folder")

WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'show_boxes' instead.

image 1/1 /content/Images/gesh_368560_311267_20.jpg: 256x256 2 GreenSpaces, 28.6ms
Speed: 0.5ms preprocess, 28.6ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)
gesh_368560_311267_20.jpg  has been saved to Masks folder
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'show_boxes' instead.

image 1/1 /content/Images/gesh_368560_311257_20.jpg: 256x256 6 GreenSpaces, 28.5ms
Speed: 0.5ms preprocess, 28.5ms inference, 1.8ms postprocess per image at shape (1, 3, 256, 256)
gesh_368560_311257_20.jpg  has been saved to Masks folder
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'show_boxes' instead.

image 1/1 /content/Images/gesh_368560_311266_20.jpg: 256x256 4 GreenSpaces, 28.8ms
Speed: 0.7ms preprocess, 28.8m

## IOU Score
If you have ground truth masks for the images you want inferences of, you can put them in `Ground_Truth` folder and run the following cell.

Note: Don't forget to create a `IOU.txt` in the `Eval_Metrices` folder

In [22]:
def calculate_iou(ground_truth, prediction):

    if not (os.path.exists(prediction)):
      print("inference mask doesn't exist")
    # Read binary masks
    mask_gt = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE)
    mask_pred = cv2.imread(prediction, cv2.IMREAD_GRAYSCALE)

    # Threshold the masks to ensure binary values
    _, mask_gt = cv2.threshold(mask_gt, 128, 255, cv2.THRESH_BINARY)
    _, mask_pred = cv2.threshold(mask_pred, 128, 255, cv2.THRESH_BINARY)

    # Compute intersection and union
    intersection = np.logical_and(mask_gt, mask_pred)
    union = np.logical_or(mask_gt, mask_pred)

    # Calculate IoU
    if np.sum(union) == 0:
      return np.sum(intersection) / (np.sum(mask_gt))
    iou = np.sum(intersection) / (np.sum(union)+0.0000007)
    return iou


ground_truth_folder_path = fr'Ground_Truth/'
prediction_path = fr"Masks/"


iou_score_list = list()
for i in os.listdir(ground_truth_folder_path):
    iou_score = calculate_iou(ground_truth_folder_path+i, prediction_path+i)
    with open('Eval_Metrices/IOU.txt', 'a') as file:
        file.write(f"{i}--> IOU SCORE: {iou_score}\n")
    print(i)
    iou_score_list.append(iou_score)

with open('Eval_Metrices/IOU.txt', 'a') as file:
    file.write(f"Arithmatic mean IoU score: {sum(iou_score_list)/len(iou_score_list)}")
print(f"Your IOUs have successfully been saved to IOU.txt file in Eval_Metrices folder\nArithmatic Mean IoU score for all images is: {sum(iou_score_list)/len(iou_score_list)}")


gesh_368560_311267_20.jpg
gesh_368560_311257_20.jpg
gesh_368560_311266_20.jpg
gesh_368560_311252_20.jpg
Your IOUs have successfully been saved to IOU.txt file in Eval_Metrices folder
Arithmatic Mean IoU score for all images is: 0.9999999999775466


## DICE Loss
If you have ground truth masks for the images you want inferences of, you can put them in `Ground_Truth` folder and run the following cell.

Note: Don't forget to create a `DICE.txt` in the `Eval_Metrices` folder

In [24]:
def calculate_dice_loss(ground_truth, prediction):
    # Read binary masks
    mask_gt = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE)
    mask_pred = cv2.imread(prediction, cv2.IMREAD_GRAYSCALE)

    # Threshold the masks to ensure binary values
    _, mask_gt = cv2.threshold(mask_gt, 128, 255, cv2.THRESH_BINARY)
    _, mask_pred = cv2.threshold(mask_pred, 128, 255, cv2.THRESH_BINARY)

    # Normalizing
    mask_gt = 1/255*mask_gt
    mask_pred = 1/255*mask_pred

    # Compute intersection and union
    intersection = np.logical_and(mask_gt, mask_pred)
    sum_add = np.sum(mask_gt) + np.sum(mask_pred)

    # Calculate IoU
    dice = 2 * np.sum(intersection) / sum_add
    return 1-dice

ground_truth_folder_path = fr'Ground_Truth/'
prediction_path = fr"Masks/"


dice_loss_list = list()
for i in os.listdir(ground_truth_folder_path):
    dice_loss = calculate_dice_loss(ground_truth_folder_path+i, prediction_path+i)
    with open('Eval_Metrices/DICE.txt', 'a') as file:
        file.write(f"{i}--> DICE LOSS: {dice_loss}\n")
    print(i)
    dice_loss_list.append(dice_loss)

with open('Eval_Metrices/DICE.txt', 'a') as file:
    file.write(f"Arithmatic mean DICE Loss: {sum(iou_score_list)/len(iou_score_list)}")
print(f"Your IOUs have successfully been saved to DICE.txt file in Eval_Metrices folder\nArithmatic Mean DICE LOSS for all images is: {sum(dice_loss_list)/len(dice_loss_list)}")


gesh_368560_311267_20.jpg
gesh_368560_311257_20.jpg
gesh_368560_311266_20.jpg
gesh_368560_311252_20.jpg
Your IOUs have successfully been saved to DICE.txt file in Eval_Metrices folder
Arithmatic Mean DICE LOSS for all images is: 0.0
